In [1]:
import os
import time
import math
import random
import socket
import json
from math import sqrt
import numpy as np
import pandas as pd
from tqdm import tqdm
from collections import defaultdict
import unicodedata
from datetime import timedelta
from google.cloud import storage

import re
import requests
import io
import copy

import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import Dataset
from torch.utils.data import DataLoader

from annoy import AnnoyIndex

In [2]:
class argparse:
    pass

def seed_everything(args):
    os.environ["PYTHONHASHSEED"] = str(args.seed)
    random.seed(args.seed)
    np.random.seed(args.seed)
    torch.manual_seed(args.seed)
    torch.cuda.manual_seed(args.seed)  # type: ignore
    torch.backends.cudnn.deterministic = True  # type: ignore
    torch.backends.cudnn.benchmark = True  # type: ignore

    return None

def init_gcs(args):
    os.environ["GOOGLE_APPLICATION_CREDENTIALS"] = args.server_data_path + args.key_name
    
    storage_client = storage.Client()
    bucket = storage_client.bucket(args.bucket_name)

    return bucket

def get_url_from_gcs(args, path, file_name, expiration = 1):
    # file_name = unicodedata.normalize('NFD', '아트웨어')
    # path = path + file_name
    return args.bucket.blob(path + file_name).generate_signed_url(version="v4", expiration = timedelta(expiration), method="GET") 

# 지우, 예림


def make_data_frame(df_biz, number, embedding, biz2common):
    number2embedding = dict(zip(number, embedding))
    df_biz['embedding'] = df_biz['출원번호'].map(number2embedding)
    df_biz['사업자번호'] = df_biz['사업자번호'].apply(lambda x: re.sub('-','',x))
    
    biznum2idx = dict(zip(biz2common['사업자번호'],biz2common['index']))
    df_biz['key'] = df_biz['사업자번호'].map(biznum2idx)
    df_biz['key'] = df_biz['key'].astype(object)
    
    return df_biz

def preprocess_dataset(df_org,mode):
    '''
    df_above: 기업에서 낸 특허가 2개 이상인 경우
    df_under : 기업에서 낸 특허가 2개 이하인 경우, 기업에서 낸 마지막 특허를 바로 ANN 알고리즘으로 찾기 
    '''
    df = copy.deepcopy(df_org)
    col = 'key' if mode == 'train' else 'identifier'
    df_above,df_under = ck_above_min(df,col)
    df_under = df_under.drop_duplicates(subset=col,keep='last')
    count_table, count_dict,corp2idx, idx2corp = make_idx_map(df_above,col)
    df_above = pd.merge(df_above,count_table[[col,'index']],on=col)
    df_above = sort_by_filing_date(df_above,col)
    return df_above,df_under,count_table,count_dict,corp2idx,idx2corp

def ck_above_min(df,col):
    temp = df[col].value_counts().reset_index()
    temp['above_min'] = temp['count'] >= 2
    df_count = pd.merge(df,temp[[col,'above_min']],on=col,how='inner')
    df_above = df_count[df_count['above_min']==True].drop('above_min',axis=1)
    df_under = df_count[df_count['above_min']==False].drop('above_min',axis=1)
    return df_above,df_under

def make_idx_map(df,col):
    count_table = df[col].value_counts().reset_index().reset_index()
    count_dict = dict(zip(count_table['index'],count_table['count']))
    key2idx = dict(zip(count_table[col],count_table['index']))
    idx2key = dict(zip(count_table['index'],count_table[col]))
    return count_table, count_dict,key2idx, idx2key
    
def sort_by_filing_date(df,col):
    return df.groupby(col).apply(lambda x:x.sort_values(by='출원일자'))

def make_seq_main(args, df, count_dict):
    def make_corp_seq(args,df):
        embedding_dict = {} 
        for idx,cnt in count_dict.items():
            seq = cnt
            part_sequence = []
            if seq >= args.max_len:
                corp_seq = df[df['index']== idx]['embedding'][-(args.max_len + 2) : ]
            else:
                corp_seq = df[df['index']== idx]['embedding']
            
            for i in range(len(corp_seq)):
                part_sequence.append(corp_seq[i])
            
            embedding_matrix = np.array(part_sequence).reshape(-1,args.d_embed)
            embedding_dict[idx] = embedding_matrix
        
        return embedding_dict
    
    corp_seq = make_corp_seq(args, df)
    
    return corp_seq

class Dataset(Dataset):
    def __init__(self, args, corp_seq, data_type="train"):
        self.args = args
        self.device = args.device
        self.corp_seq = corp_seq
        self.data_type = data_type
        self.max_len = args.max_len
        self.d_embed = args.d_embed
        
    def __len__(self):
        return len(self.corp_seq)
    
    def __getitem__(self, index):

        corp_id = index
        patents = self.corp_seq[index] # index번째 기업의 sequence 

        if self.data_type == "train":
            input_ids = patents[:-2]
            target_pos = patents[1:-1]
            answer = [0] # no use

        elif self.data_type == "valid":
            input_ids = patents[:-1]
            target_pos = patents[1:]
            # answer = [patents[-1]]
            answer = np.array([patents[-1]])
        else:
            input_ids = patents[:]
            target_pos = patents[:]  # will not be used
            answer = []
        
        mask = np.ones(self.max_len)
        pad_len = self.max_len - len(input_ids)
        if pad_len > 0 :
            pad = np.zeros((pad_len,self.d_embed))
            input_ids = np.concatenate((pad,input_ids),axis=0)
            target_pos = np.concatenate((pad,target_pos),axis=0)

            mask[:pad_len] = 0
            
        input_ids = input_ids[-self.max_len :]
        target_pos = target_pos[-self.max_len :]

        assert len(input_ids) == self.max_len
        assert len(target_pos) == self.max_len

        cur_tensors = (
            torch.tensor(corp_id, device=self.device),  # user_id for testing
            torch.tensor(mask, device=self.device),  # user_id for testing
            torch.tensor(input_ids,dtype = torch.float, device=self.device),
            torch.tensor(target_pos,dtype = torch.float, device=self.device),
            torch.tensor(answer,dtype = torch.float, device=self.device),
        )

        return cur_tensors
    
class PositionalEncoding(nn.Module):
    def __init__(self, d_model, max_len):
        super(PositionalEncoding, self).__init__()
        self.encoding = torch.zeros(max_len, d_model)
        self.encoding.requires_grad = False
        position = torch.arange(0, max_len, dtype=torch.float).unsqueeze(1)
        div_term = torch.exp(torch.arange(0, d_model, 2).float() * (-math.log(10000.0) / d_model))
        self.encoding[:, 0::2] = torch.sin(position * div_term)
        self.encoding[:, 1::2] = torch.cos(position * div_term)
        self.encoding = self.encoding.unsqueeze(0)

    def forward(self, x):
        device = x.device
        return x + self.encoding[:, :x.size(1)].detach().to(device)

class MultiHeadAttention(nn.Module):
    def __init__(self, hidden_dim, num_heads, dropout):
        super(MultiHeadAttention, self).__init__()
        self.h = num_heads # 병렬 attention head 개수
        self.head_dim = hidden_dim // num_heads
        # self.d_model = hidden_dim * num_heads ##
        self.d_model = hidden_dim ##
        
        self.Q_fc = nn.Linear(hidden_dim, hidden_dim, bias=False)
        self.K_fc = nn.Linear(hidden_dim, hidden_dim, bias=False)
        self.V_fc = nn.Linear(hidden_dim, hidden_dim, bias=False)
        self.O_fc = nn.Linear(hidden_dim, hidden_dim, bias=False)
        
        self.dropout = nn.Dropout(dropout)
        self.layerNorm = nn.LayerNorm(hidden_dim, 1e-6)
        
    def forward(self, Q, K, V, mask=None):
        n_batch = Q.size(0)
        residual = Q
        
        def transform(x, fc):
            out = fc(x)
            out = out.view(n_batch, -1, self.h, self.head_dim)
            out = out.transpose(1, 2)
            return out
        
        Q = transform(Q, self.Q_fc)
        K = transform(K, self.K_fc)
        V = transform(V, self.V_fc)
        d_k = K.shape[-1]
        device = torch.device('cuda')
        attention_score = torch.matmul(Q, K.transpose(-2, -1))
        attention_score = attention_score / torch.sqrt(torch.tensor(d_k, dtype=torch.double, device=device))
            
        if mask is not None:
            attention_score = attention_score.masked_fill(mask == 0, -1e12)
        
        attention_score = F.softmax(attention_score, dim=-1)
        out = torch.matmul(attention_score, V)
        out = out.transpose(1, 2).contiguous().view(n_batch, -1, self.d_model)
        out = self.O_fc(out)
        out = self.dropout(out) + residual
        return self.layerNorm(out)
    
class PointWiseFeedForward(nn.Module):
    def __init__(self, hidden_dim, dropout):
        super(PointWiseFeedForward, self).__init__()
        self.hidden_size = hidden_dim
        self.fc1 = nn.Linear(self.hidden_size, self.hidden_size * 4)
        self.activation = lambda x: x * 0.5 * (1.0 + torch.erf(x / sqrt(2.0)))
        self.fc2 = nn.Linear(self.hidden_size * 4, self.hidden_size)
        self.dropout = nn.Dropout(dropout)
        self.layerNorm = nn.LayerNorm(self.hidden_size, 1e-6)
        
    def forward(self, x):
        residual = x
        out = self.fc1(x)
        out = self.activation(out)
        out = self.fc2(out)
        out = self.dropout(out) + residual
        return out

class SASRecBlock(nn.Module):
    def __init__(self, hidden_dim, num_heads, dropout):
        super(SASRecBlock, self).__init__()
        self.self_attention = MultiHeadAttention(hidden_dim, num_heads, dropout)
        self.pointwise_feed_forward = PointWiseFeedForward(hidden_dim, dropout)

    def forward(self, x, mask):
        out = self.self_attention(x, x, x, mask)
        out = self.pointwise_feed_forward(out)
        return out
    
class SASRec(nn.Module):
    def __init__(self, max_seq_length, hidden_dim, num_heads, num_blocks, dropout):
        super(SASRec, self).__init__()
        self.positional_encoding = PositionalEncoding(hidden_dim, max_seq_length)
        self.sas_blocks = nn.ModuleList([SASRecBlock(hidden_dim, num_heads, dropout) for _ in range(num_blocks)])
        self.dropout = nn.Dropout(dropout)
        self.layer_norm = nn.LayerNorm(hidden_dim, eps=1e-6)

        self.initializer_range = args.initializer_range
        self.apply(self.init_weights)
        
    def make_pad_mask(self, x, pad_idx=0): ##
        max_seq_length = x.size(-1)

        row_wise = x.ne(pad_idx).unsqueeze(1).unsqueeze(3)
        row_wise = row_wise.repeat(1, 1, 1, max_seq_length)

        column_wise = x.ne(pad_idx).unsqueeze(1).unsqueeze(2)
        column_wise = column_wise.repeat(1, 1, max_seq_length, 1)

        pad_mask = row_wise & column_wise
        pad_mask.requires_grad = False

        return pad_mask
    
    def make_subsequent_mask(self, x, pad_idx=0): ##
        max_seq_length = x.size(-1)

        attention_shape = (1, max_seq_length, max_seq_length)
        subsequent_mask = torch.triu(torch.ones(attention_shape), diagonal=1)
        subsequent_mask = (subsequent_mask == pad_idx).unsqueeze(1)
        subsequent_mask.requires_grad = False

        return subsequent_mask
    
    def init_weights(self, module):
        if isinstance(module, (nn.Linear, nn.Embedding)):
            module.weight.data.normal_(mean=0.0, std=self.initializer_range)
        elif isinstance(module, nn.LayerNorm):
            module.bias.data.zero_()
            module.weight.data.fill_(1.0)
        if isinstance(module, nn.Linear) and module.bias is not None:
            module.bias.data.zero_()
    
    def forward(self, input_seq, mask=None):
        # Embedding and positional encoding
        seqs = input_seq
        seqs += self.positional_encoding(seqs)

        mask_pad = self.make_pad_mask(mask).to(seqs.device) ## 
        mask_time = self.make_subsequent_mask(mask).to(seqs.device) ##
        mask = mask_pad & mask_time

        # print(mask.shape)

        # Transformer blocks
        for sas_block in self.sas_blocks: 
            seqs = sas_block(seqs, mask)
        output = seqs
        return output
    
def setting_model(args):
    model = SASRec(args.max_len, args.d_embed, args.num_heads, args.num_layers, args.dropout_rate).to(args.device)
    # 모델의 가중치 파라미터를 Double 형식으로 설정
    model.apply(lambda module: setattr(module, 'dtype', torch.double))
    return model

class Annoy:
    def __init__(self, args, annoy_index):
        self.n_trees = args.n_trees
        self.n = args.n
        self.d_embed = args.d_embed
        self.annoy_index = annoy_index
        
    def find_annoy(self, vector):
        ann_idx, ann_score = self.annoy_index.get_nns_by_vector(vector, self.n, include_distances=True)
        return ann_idx[::-1],ann_score[::-1]

class Inference:
    def __init__(self,args, key, transformer, annoy_index, count_dict):
        self.args = args
        self.n_trees = self.args.n_trees
        self.n = self.args.n
        self.d_embed = self.args.d_embed
        self.key = key
        self.model = transformer
        self.annoy_index = annoy_index
        self.device = self.args.device
        
    def inference(self):
        device = torch.device(self.device)
        idx = corp2idx[self.key]
        inference_dataset = Dataset(args, corp_seq, data_type="inference")[idx]
        inference_dataloader = DataLoader(inference_dataset, batch_size= 1)
        with torch.no_grad():
            self.model.eval()
            corp_id, mask, input_seq, target_pos, _ = inference_dataloader
            mask = mask.to(device)
            input_seq, target_pos = input_seq.to(device), target_pos.to(device)
            output = self.model(input_seq, mask=mask)
            output = output[:, -1, :].cpu().data.numpy()
        output = output.reshape(-1,1)
        return output
    
    def scoring(self):
        self.inference_vector = self.inference()
        annoy = Annoy(args, self.annoy_index)
        ann_idx,ann_score = annoy.find_annoy(self.inference_vector) # (768, 1)
        score_dict = {}
        for idx,score in zip(ann_idx,ann_score):
            key = df_biz.iloc[idx]['key']
            if key not in score_dict:
                score_dict[key] = [1,score]
            else:
                score_dict[key][0] += 1
                
        return score_dict
    
    def find_candidates(self):
        score_dict = self.scoring()
        k = len(score_dict)
        # corp_list = np.repeat(df_biz[df_biz['key']==self.key]['회사명'].iloc[0],k)
        corp_list = np.repeat(key2corpname[self.key],k)
        key_list = np.repeat(self.key, k)
        sorted_score = sorted(score_dict.items(), key=lambda x: (x[1][0],x[1][1]), reverse=True)
        target_corp_list, target_key_list = [],[]
        for target_corp_info in sorted_score:
            target_corp_key = target_corp_info[0]
            target_corp = key2corpname[target_corp_key]
            target_corp_list.append(target_corp)
            target_key_list.append(target_corp_key)
        df_candidate = pd.DataFrame({'의뢰기업':corp_list,'의뢰기업_key':key_list,'대상기업':target_corp_list,'대상기업_key':target_key_list})
        return df_candidate

In [3]:
args = argparse()

args.seed = 42
seed_everything(args)

args.server_home_path = '.'
args.server_data_path = args.server_home_path + '/data'

args.gcp_home_path = ''
args.gcp_data_path = args.gcp_home_path
args.gcp_imgs_path = args.gcp_data_path + 'imgs/'

args.key_name = '/linear-yen-393604-4b0eb55e48b1.json'
args.bucket_name = 'exit_bucket'
args.patent_name = 'patent_companies.tsv' 
args.app_number_name = 'number.npy'
args.embedding_name = 'embedding.npy'
args.common_name = 'biz2common.tsv'

args.transformer_path = args.gcp_data_path + 'transformer/'
args.transformer_version = 'version1'
args.transformer_file_name = f'transformer_{args.transformer_version}.pt'
args.d_embed=768
args.max_len = 1024
args.initializer_range = 0.02
args.num_heads = 4
args.num_layers = 2
args.dropout_rate = 0.2
args.device = "cuda" if torch.cuda.is_available() else "cpu"

args.ann_path = args.server_data_path + '/ann'
args.ann_version = 'version1'
args.ann_file_name = f'/ann_{args.ann_version}.annoy'
args.n_trees = 20
args.n = 100

args.valuation_path = args.gcp_data_path + 'valuation/'
args.valuation_version = 'version1'
args.valuation_file_name = f'valuation_{args.valuation_version}.csv'

In [4]:
start = time.time()
args.bucket = init_gcs(args)

url = get_url_from_gcs(args, args.gcp_data_path, args.patent_name)
response = requests.get(url)
df_biz = pd.read_csv(io.BytesIO(response.content), sep = '\t', parse_dates=['출원일자'])
df_biz = df_biz[df_biz['출원일자'].notna()]
print(time.time() - start)

url = get_url_from_gcs(args, args.gcp_data_path, args.app_number_name)
response = requests.get(url)
number = np.load(io.BytesIO(response.content), allow_pickle = True)
print(time.time() - start)

url = get_url_from_gcs(args, args.gcp_data_path, args.embedding_name)
response = requests.get(url)
embedding = np.load(io.BytesIO(response.content), allow_pickle = True)
print(time.time() - start)

url = get_url_from_gcs(args, args.gcp_data_path, args.common_name)
response = requests.get(url)
biz2common = pd.read_csv(io.BytesIO(response.content), sep = '\t', dtype={'사업자번호':'object','index':'object'})
print(time.time() - start)

df = make_data_frame(df_biz, number, embedding, biz2common)
biz2idx = dict(zip(df['사업자번호'],df['key']))
idx2biz = dict(zip(df['key'],df['사업자번호'].apply(lambda x : f'{x[:3]}-{x[3:5]}-{x[5:]}')))
df_above, df_under, count_table, count_dict, corp2idx, idx2corp = preprocess_dataset(df, 'train') ## 1개짜리는..?
args.num_items = len(count_dict)
corp_seq = make_seq_main(args, df_above, count_dict)
key2corpname = dict(zip(biz2common['index'], biz2common['회사명']))
print(time.time() - start)

70.77718949317932
72.11422228813171
164.63167023658752
164.98443794250488
261.459237575531


In [ ]:
# 서버 IP 및 열어줄 포트
HOST = ''
PORT = 30005
PREV = dict()

serverSock = socket.socket(socket.AF_INET, socket.SOCK_STREAM)
serverSock.bind(('', PORT))
serverSock.listen(1)

# try:
while True:
    try:
        print(f'{PORT}번 포트로 접속 대기중...')
        connectionSock, addr = serverSock.accept()

        print(str(addr), '에서 접속되었습니다.')

        # -------------------------------
        recvData = connectionSock.recv(1024)
        recvData = json.loads(recvData.decode('utf-8'))
        print(recvData)

        if 'transformer_version' in recvData.keys():
        
            args.transformer_version = recvData['transformer_version']
            args.ann_version = recvData['ann_version']
            args.valuation_version = recvData['valuation_version']

            if 'transformer_version' not in PREV.keys() or PREV['transformer_version'] != args.transformer_version:
                PREV['transformer_version'] = args.transformer_version
                args.transformer_file_name = f'transformer_{args.transformer_version}.pt'

                url = get_url_from_gcs(args, args.transformer_path, args.transformer_file_name)
                response = requests.get(url)
                transformer_path = io.BytesIO(response.content)

                transformer = setting_model(args)
                transformer.load_state_dict(torch.load(transformer_path))

            if 'ann_version' not in PREV.keys() or PREV['ann_version'] != args.ann_version:
                PREV['ann_version'] = args.ann_version
                args.ann_file_name = f'/ann_{args.ann_version}.annoy'

                annoy_index = AnnoyIndex(args.d_embed, 'angular') # batch로 돈다고 가정해야 할듯 함
                annoy_index.load(args.server_data_path + args.ann_file_name)

            if 'valuation_version' not in PREV.keys() or PREV['valuation_version'] != args.valuation_version:
                PREV['valuation_version'] = args.valuation_version
                args.valuation_file_name = f'valuation_{args.valuation_version}.csv'

                url = get_url_from_gcs(args, args.valuation_path, args.valuation_file_name)
                response = requests.get(url)
                valuation = pd.read_csv(io.BytesIO(response.content))
                valuation['index'] = valuation['index'].astype(str)

            sendData = 'Completed'
            connectionSock.send(sendData.encode('utf-8'))
            

        elif 'business_number' in recvData.keys():
            args.business_number = biz2idx[recvData['business_number'].replace('-', '')]

            inference = Inference(args, args.business_number, transformer, annoy_index, count_dict)
            inference_return = inference.find_candidates()

            inference_return = inference_return.merge(valuation[['index', 'value']], left_on = '의뢰기업_key', right_on = 'index', how = 'left').drop('index', axis = 1).rename(columns = {'value' : '의뢰기업_value'})
            inference_return = inference_return.merge(valuation[['index', 'value']], left_on = '대상기업_key', right_on = 'index', how = 'left').drop('index', axis = 1).rename(columns = {'value' : '대상기업_value'})

            inference_return['의뢰기업_biz'] = inference_return['의뢰기업_key'].map(idx2biz)
            inference_return['대상기업_biz'] = inference_return['대상기업_key'].map(idx2biz)

            inference_return = inference_return[inference_return['의뢰기업_value'] > inference_return['대상기업_value']]
            inference_return = inference_return[:10]
            
            temp_v1 = dict(zip(inference_return['의뢰기업'], inference_return['의뢰기업_value']))
            temp_k1 = dict(zip(inference_return['의뢰기업'], inference_return['의뢰기업_key']))
            temp_b1 = dict(zip(inference_return['의뢰기업'], inference_return['의뢰기업_biz']))

            temp_v2 = dict(zip(inference_return['대상기업'], inference_return['대상기업_value']))
            temp_k2 = dict(zip(inference_return['대상기업'], inference_return['대상기업_key']))
            temp_b2 = dict(zip(inference_return['대상기업'], inference_return['대상기업_biz']))

            result = []
            for key in temp_v1.keys():
                result.append(key)
                result.append(str(temp_v1[key]))
                result.append(str(temp_k1[key]))
                result.append(str(temp_b1[key]))

            for key in temp_v2.keys():
                result.append(key)
                result.append(str(temp_v2[key]))
                result.append(str(temp_k2[key]))
                result.append(str(temp_b2[key]))

            sendData = ' '.join(result)
            connectionSock.send(sendData.encode('utf-8'))

        connectionSock.close()

    except Exception as e:
        print('Error :', e)
        connectionSock.close()
serverSock.close()

30005번 포트로 접속 대기중...
('182.218.59.27', 53172) 에서 접속되었습니다.
{'transformer_version': 'version1', 'ann_version': 'version1', 'valuation_version': 'version1'}
30005번 포트로 접속 대기중...
('182.218.59.27', 53176) 에서 접속되었습니다.
{'business_number': '125-86-25094'}
30005번 포트로 접속 대기중...
('182.218.59.27', 53186) 에서 접속되었습니다.
{'transformer_version': 'version1', 'ann_version': 'version1', 'valuation_version': 'version1'}
30005번 포트로 접속 대기중...
('182.218.59.27', 53187) 에서 접속되었습니다.
{'business_number': '125-86-25094'}
30005번 포트로 접속 대기중...
